### Note: 
This notebook is the same as concepts_based_solution.ipynb but with slightly different prompts for the answer compilation part, and using gpt-3.5-turbo for query generation. This notebook matches the code used in the StreamLit app.

# ChatGent source code

This notebook contains the the code of each step in the workflow designed to generate responses to users' questions in ChatGent. The code is accompanied with brief explanation as well as possible future improvements here and there.

### Importing necessary libraries & defining variables

The dataset worked with for this project is only the Decisions dataset. Decisions made by the city of Gent are stored in a SparQL endpoint from the previous PROBE project by District09. Example questions and queries are made in a json, as well as labels of decisions and the labels' URIs.

In [1]:
import json
import requests
from openai import OpenAI
import re
from datetime import date

# Define the SparQL endpoint
endpoint_url = "https://probe.stad.gent/sparql"

# Just to have the AI double check its answer for "when/wanneer" questions
today = date.today()

# Define the API key as an environment variable and this should work
client = OpenAI()

# Loading the example questions and queries to be used as context in the prompt for generating queries
with open('../json_files/questions_and_queries.json', 'r', encoding='utf-8') as file:
    example_data = json.load(file)

# Loading the annotations of decisions: labels and their URIs
with open('../json_files/annotations.json', 'r', encoding='utf-8') as file:
    label_data = json.load(file)

### Function to generate SparQL query
The following function uses gpt-3.5-turbo to translate the user's question to a SparQL query. Several instructions are given in the prompt and examples are also provided. This has proven to help the AI generate better and more correct/accurate queries.

In [2]:
def generate_sparql_query(user_question, label_data, examples_data):
    concepts_and_labels = "\n".join(
        f"URI of Label: {pair['uri']}\nLabel: {pair['label']}\n" for pair in label_data
    )
    example_queries = "\n".join(
        f"User Question: {pair['user_question']}\nSPARQL Query: {pair['sparql_query']}\n" for pair in examples_data
    )

    prompt = f"""

    GIVE ONLY THE QUERY AS AN ANSWER TO THE FOLLOWING PROMPT:

    The following are all the labels for the decisions in the decisions dataset and their URIs:

    {concepts_and_labels}

    Based on the user's question: {user_question}, go through all the labels then choose all possible labels that best matches the question's theme and context.

    NEXT STEP:

    The following are examples of user questions in Dutch and their corresponding SPARQL queries:

    {example_queries}

    Based on the examples above and the URIs of the chosen labels, generate a SPARQL query for the following user question:

    User Question: {user_question}
    SPARQL Query:

    But please make sure to use the URIs of the chosen labels in the "?annotation oa:hasBody" part of the query like in the examples. If there is more than one label chosen query for them using the same structure in the second example query in {example_queries}. This utilizes VALUES.
    
    If the user doesn't set a limit to the number of decisions they want to see, limit them to 3.

    Always choose the most recent decision (by ordering on publication date "eli:date_publication") unless prompted otherwise by the user.
    
    Don't add '`' as you wish.

    Then after filtering on label, add a filter for the title or description or motivering with keywords that you extract from the question. (NOTE: Do not use generic words that apply to all decisions about Gent, like 'Gent', as a keyword).

    """
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            #model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that generates SPARQL queries to be run on a SPARQL endpoint containing data about decisions of the city of Gent based on user questions and labels of decisions related to their questions. Your language is Dutch."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0
        )

    return response

### Checking/Re-writing SparQL query
In some cases, there might be decisions related to the user's question as in they fall under its category and could be useful for the user, but get filtered out when querying for decisions within this theme with certain keywords. This is because keyword matching is literal. For example, if the query is looking for the keyword "zwembad" and a decision is about swimming but doesn't contain that word specifically (instead it has "zwemmen"), it will filter out that decision. Another scenario is if the AI uses generic words that apply to any decision as keywords, like 'gent'. Although it's instructed to not do so in the prompt, just in case, we have the keyword matching removed from the query when no results are found. This way we only stick to filtering on labels of decisions.

The following function has an AI model remove any keyword filtering and give back a proper SparQL query.

In [3]:
def check_sparql_query(query):

    prompt = f"""
    If the query generated {query} contains looking for keywords, generate the same SPARQL query but remove the keywords filtering.
    Don't add '`' to the query as you wish.
    """
    response_2 = client.chat.completions.create(
            model="gpt-3.5-turbo",
            #model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a SPARQL query refiner."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0
        )

    return response_2

### Function to run the query

The following function is used to run the query on the SparQL endpoint using an http request. It returns a list of all queried decisions.

In [4]:
def run_query(query):
    headers = {
        "Accept": "application/sparql-results+json"
    }
    params = {
        "query": query
    }

    response = requests.get(endpoint_url, headers=headers, params=params)
    results = None
    if response.status_code == 200:
        results = response.json()
        #print(f"Results for question '{user_question}':", results)
    else:
        #print(f"Failed to execute query for question '{user_question}':", response.status_code)
        return []
    
    if results is None:
        return []
    
    results_content = results['results']['bindings']

    cleaned_decisions = []

    for decision in results_content:
        cleaned_decision = {}
        for key, detail in decision.items():
            # Extract the value and remove any \n or extra spaces
            cleaned_value = re.sub(r'\s+', ' ', detail['value']).strip()
            cleaned_decision[key] = cleaned_value
        cleaned_decisions.append(cleaned_decision)

    return cleaned_decisions

# Running the main program
A test user question is defined, then the function to generate a SparQL query is run using that question.
The user question and SparQL query are printed out.

In [5]:
# Example user question
user_question = "Waar kan ik gaan zwemmen?"
# user_question = "Where can I go swimming?"

# Generate SPARQL query for the example user question
sparql_query = generate_sparql_query(user_question, label_data=label_data, examples_data=example_data)
print(f"User Question: {user_question}")
print(f"Generated SPARQL Query: {sparql_query}")

User Question: Waar kan ik gaan zwemmen?
Generated SPARQL Query: ChatCompletion(id='chatcmpl-9rnPKIl2bo9PNXErraqBJlw8S4Rsv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="PREFIX dct: <http://purl.org/dc/terms/>\nPREFIX prov: <http://www.w3.org/ns/prov#>\nPREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>\nPREFIX eli: <http://data.europa.eu/eli/ontology#>\nPREFIX oa: <http://www.w3.org/ns/oa#>\nSELECT ?annotation ?target ?title ?motivering (GROUP_CONCAT(?value; separator=', ') AS ?values) ?description ?date ?derivedFrom\nWHERE {\n    ?annotation oa:hasBody <http://stad.gent/id/concepts/gent_words/485> ;\n               oa:hasTarget ?target .\n    ?target eli:title ?title ;\n           besluit:motivering ?motivering ;\n           prov:value ?value ;\n           eli:description ?description ;\n           eli:date_publication ?date ;\n           prov:wasDerivedFrom ?derivedFrom .\n    FILTER (CONTAINS(LCASE(?title), 'zwemmen') || 

The following code snippet is just to clean the resulted query from any potential extra text generated by the AI before the query, and to remove new lines.

In [6]:
query_content = sparql_query.choices[0].message.content

prefix_position = query_content.find("PREFIX")
if prefix_position != -1:
    query_content = query_content[prefix_position:]

In the example we chose, there weren't any decisions retrieved using the generated SparQL query which had contained filtering on keywords i.e. "FILTER (CONTAINS(LCASE(?title), 'middelbare school')".

In [7]:
cleaned_decisions = run_query(query_content)
print(cleaned_decisions)

[{'values': 'Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen. , Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.', 'title': '2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen', 'target': 'https://data.gent.be/id/besluiten/22.0518.8364.2858', 'motivering': 'Motivering Wat gaat aan deze beslissing vooraf Tijdens het weekend van 14-15 mei was zwemmen aan de Blaarmeersen ondanks de hoge temperaturen niet mogelijk. Het bijzonder goede weer w

In [9]:
cleaned_decisions

[{'values': 'Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen. , Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.',
  'title': '2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen',
  'target': 'https://data.gent.be/id/besluiten/22.0518.8364.2858',
  'motivering': 'Motivering Wat gaat aan deze beslissing vooraf Tijdens het weekend van 14-15 mei was zwemmen aan de Blaarmeersen ondanks de hoge temperaturen niet mogelijk. Het bijzonder goede 

The retrieved decisions list is checked. If it's empty, the check_sparql_query function is called. It will remove an filtering on keywords and give back a new query. We run the new query and print the results list which in this time is not empty.

In [10]:
if not cleaned_decisions:
    sparql_query_2 = check_sparql_query(query_content)
    query_content_2 = sparql_query_2.choices[0].message.content
    prefix_position = query_content.find("PREFIX")
    if prefix_position != -1:
        query_content_2 = query_content_2[prefix_position:]

    print("SECOND SPARQL:", query_content_2)

    cleaned_decisions = run_query(query_content_2)
    print("AGAIN:", cleaned_decisions)

In [11]:
cleaned_decisions

[{'values': 'Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen. , Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.',
  'title': '2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen',
  'target': 'https://data.gent.be/id/besluiten/22.0518.8364.2858',
  'motivering': 'Motivering Wat gaat aan deze beslissing vooraf Tijdens het weekend van 14-15 mei was zwemmen aan de Blaarmeersen ondanks de hoge temperaturen niet mogelijk. Het bijzonder goede 

In order to maintain transparency with the citizens, we provide them with links to the decisions used to generate the answer, these are saved as resources below:

In [12]:
resources = []
resources_names = []

# Print or use the cleaned_decisions as needed
for d in cleaned_decisions:
    print(d)

for d in cleaned_decisions:
    resources.append(d['derivedFrom'])
    resources_names.append(d['title'])

{'values': 'Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen. , Artikel 1 De gemeenteraad draagt het college van burgemeester en schepenen op om samen met Farys een flexibele zwemregeling voor de Blaarmeersen te laten onderzoeken (inclusief financiële en operationele impact), zodat er ook vroeger tijdens het voorseizoen kan gezwommen worden op warme dagen.', 'title': '2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen', 'target': 'https://data.gent.be/id/besluiten/22.0518.8364.2858', 'motivering': 'Motivering Wat gaat aan deze beslissing vooraf Tijdens het weekend van 14-15 mei was zwemmen aan de Blaarmeersen ondanks de hoge temperaturen niet mogelijk. Het bijzonder goede weer wa

The final step is to compile the answer which will be shown to the user using the retrieved decisions' details. A prompt is made with the instructions that the LLM should take into account when generating the response:

In [13]:
prompt_2 = f"""
ELEMENTS TO USE:
1. User's question: {user_question}
2. Data to use for answer: {cleaned_decisions}

YOUR ROLE: 
- You answer the user's question. However, chat normally if the user's prompt is not a question.
- You answer in the language the user asked in. For example,if they ask in English, you answer in English, and if they ask in Dutch, you answer in Dutch.

INSTRUCTIONS:
- If it is a question, generate a response using ONLY the given data.
- The date today is {today}. Answer with that in mind.
- If you do not have data, the only external links you are allowed to refer to when looking for an answer are:
    - https://stad.gent
    - https://gentsefeesten.stad.gent
- If the question is asking about a date of an event and the data provided is from the past, say you do not know.

REQUIREMENTS YOU MUST FOLLOW:
1. If the retrieved decisions' dates do not match the current date's year: {today}, you must tell the user that it's not recent.
2. If you don't have an answer, you are only allowed to refer the user to https://stad.gent website.
3. You are only allowed to use the provided data to compile the answer.
4. Use the word "besluiten" when referring to decisions in the Dutch language. Don't use "beslissingen".
5. Be friendly and use plain language, avoiding bureaucratic terms.

LAST IMPORTANT STEPS YOU MUST FOLLOW: Before showing your answer, ensure it matches the user's question:
- If it relates but doesn't answer exactly, mention: "It might relate but isn't necessarily the answer you want."
- Provide the resources in bullet points: {resources}, formatted with the {resources_names} as links ({resources}).

"""

completion_2 = client.chat.completions.create(
    #model="gpt-3.5-turbo",
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant for the citizens of Gent when they ask a question on the city's website regarding the decisions made by the city. Be friendly, speak in easy to use terms. You use both the user's question and relevant decisions passed to you."},
        {"role": "user", "content": prompt_2}
    ],
    temperature=0
)

msg_2 = completion_2.choices[0].message.content

msg_2

'Je vraag over waar je kunt gaan zwemmen is heel relevant! Op dit moment is de belangrijkste plek om te zwemmen in Gent de Blaarmeersen. De gemeenteraad heeft besloten om samen met Farys een flexibele zwemregeling te onderzoeken, zodat er ook eerder in het seizoen gezwommen kan worden op warme dagen. Dit is bedoeld om ervoor te zorgen dat je op mooie dagen, zelfs voor het officiële zwemseizoen, kunt genieten van het water.\n\nHoud er rekening mee dat deze besluiten uit 2022 zijn en dus niet recent zijn. Voor de meest actuele informatie over zwemmen in Gent, kun je de website van de stad bezoeken.\n\nHier zijn enkele relevante besluiten die je kunt bekijken:\n- [2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)\n- [2022_AM_00005 - Amendement van het college van burgemeester en schepenen op het voorstel van raadsbesluit, ingediend door Anneleen Van Bos

In [14]:
print(msg_2)

Je vraag over waar je kunt gaan zwemmen is heel relevant! Op dit moment is de belangrijkste plek om te zwemmen in Gent de Blaarmeersen. De gemeenteraad heeft besloten om samen met Farys een flexibele zwemregeling te onderzoeken, zodat er ook eerder in het seizoen gezwommen kan worden op warme dagen. Dit is bedoeld om ervoor te zorgen dat je op mooie dagen, zelfs voor het officiële zwemseizoen, kunt genieten van het water.

Houd er rekening mee dat deze besluiten uit 2022 zijn en dus niet recent zijn. Voor de meest actuele informatie over zwemmen in Gent, kun je de website van de stad bezoeken.

Hier zijn enkele relevante besluiten die je kunt bekijken:
- [2022_VVB_00015 - Geamendeerd voorstel tot raadsbesluit van raadslid Anneleen Van Bossuyt: Zwemmen aan de Blaarmeersen](https://ebesluitvorming.gent.be/zittingen/21.1004.3056.7946/notulen)
- [2022_AM_00005 - Amendement van het college van burgemeester en schepenen op het voorstel van raadsbesluit, ingediend door Anneleen Van Bossuyt, b

In cases where there are no retrieved decisions from the query, the compiled answer states that it couldn't find anything. It is not meant to look at external resources on its own (this is included as an instruction in the prompt). This is to avoid hallucination.